Let's go the starting point once again...

In [274]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

contents = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()
soup = BeautifulSoup(contents, "lxml")

In [275]:
table = soup.find('table')
tb = table.find('tbody')
linhas = tb.find_all('tr')
lista =[]
for linha in linhas:  
    elementos = linha.find_all('td')
    listinha =[]
    for elemento in elementos:
        listinha.append(elemento.string)
    lista.append(listinha)

df = pd.DataFrame(lista, columns=['Postcode','Borough','Neighbourhood'])

In [276]:
df =df.dropna()

df['Neighbourhood']= df['Neighbourhood'].str.rstrip(']\n')
df['Borough']=df['Borough'].str.rstrip()
df = df[(df != "Not assigned").all(axis =1)]
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join)
df = pd.DataFrame({"Postcode":df.index.get_level_values("Postcode"), "Borough":df.index.get_level_values("Borough"), "Neighbourhood":df.values})

In [277]:
!pip install geocoder

distributed 1.21.8 requires msgpack, which is not installed.


In [278]:
import geocoder
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

In [279]:
latitude=[]
longitude=[]
for postal_code in df['Neighbourhood']:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

In [280]:
df['Latitude'] = latitude
df['Longitude'] = longitude

--All right, time to do some clustering!--

first, import everything!

In [281]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Geting Toronto informations

In [282]:
address = 'Toronto, TO'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.638093, -79.4665843.


Graphically:

In [283]:
mapTo = folium.Map(location=[latitude, longitude], zoom_start=10)

for index,mark in df.iterrows():    
    label = '{}, {}'.format(mark['Neighbourhood'], mark['Borough'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [mark['Latitude'], mark['Longitude']],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapTo)  
    
mapTo

Making the searches to get info about the venues close to each postcode location (within 250m radius, max = 100) 


In [284]:
results =[]
for index,mark in df.iterrows():    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id=C5VXMKLZE034BL2ZUY5BXXBDSHXPM2GUAKR3D2N4MI45BAIB&client_secret=REL0OJF0KSQGYLBEQBBDAKEIMEWKUZKRNDVHX3GNQ0HJ30PC&v=20180605&ll={},{}&radius=500&limit=100'.format(
    mark['Latitude'],
    mark['Longitude'] )
    result = requests.get(url).json()    
    results.append([mark['Neighbourhood'], mark['Latitude'], mark['Longitude'] , result])

In [285]:
nearVenues = []
for local in results: 
    for group in local[3]['response']['groups']:
        for item in group['items']:  
            for category in item['venue']['categories']:
                nearVenues.append( {
                    'neighborhood': local[0],
                    'latitude':local[1],
                    'longitude':local[2],
                    'location': local[3]['response']['headerFullLocation'], 
                    'name': item['venue']['name'],
                    'type': category['name'] 
                } )


In [286]:
nearVenuesDF = pd.DataFrame(nearVenues)

With this, we hava a dataframe with the neighborhood and it's nearest venues, which is have the following features:

In [287]:
nearVenuesDF.head()

,latitude,location,longitude,name,neighborhood,type
0,43.752743,"Scarborough Village, Toronto",-79.192777,Via. Rail 651,Guildwood,Moving Target
1,43.752743,"Scarborough Village, Toronto",-79.192777,Poplar,Guildwood,Baseball Field
2,43.752743,"Scarborough Village, Toronto",-79.192777,VIA Rail Guildwood,Guildwood,Train Station
3,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Mama's Boys Burgers,East Birchmount Park,Burger Joint
4,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Shell,East Birchmount Park,Gas Station


In [288]:
nearVenuesDF.shape

(1604, 6)

In [289]:
nearVenuesDF.groupby('neighborhood').size().reset_index()

,neighborhood,0
0,Albion Gardens,6
1,"Bathurst Quay, Island airport, Harbourfront West",26
2,"Bloordale Gardens, Eringate, Old Burnhamthorpe",7
3,Brockton,51
4,Business reply mail Processing Centre969 Eastern,39
5,Caledonia-Fairbanks,7
6,Canada Post Gateway Processing Centre,17
7,Central Bay Street,100
8,Christie,15
9,"Clarks Corners, Sullivan",100


Now let's get the frequency of every venue and sort then, getting the top 10

In [290]:
dFinal = pd.get_dummies(nearVenuesDF[['type']], prefix="", prefix_sep="")
dFinal['Neighborhood']= nearVenuesDF['neighborhood']
dFinal = dFinal.groupby('Neighborhood').sum().reset_index()
dFinal.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,...,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Albion Gardens,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Bathurst Quay, Island airport, Harbourfront West",0,1,1,2,2,2,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Bloordale Gardens, Eringate, Old Burnhamthorpe",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Brockton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,0,1,0,0,0
4,Business reply mail Processing Centre969 Eastern,0,0,0,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0


And proceed to order the venues in a data frame

In [291]:
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(10):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dFinal['Neighborhood']

for ind in np.arange(dFinal.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dFinal.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albion Gardens,Caribbean Restaurant,Pharmacy,Supermarket,Bank,Asian Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Electronics Store
1,"Bathurst Quay, Island airport, Harbourfront West",Coffee Shop,Park,Café,Airport Lounge,Airport Service,Airport Terminal,Japanese Restaurant,Sushi Restaurant,Caribbean Restaurant,Dance Studio
2,"Bloordale Gardens, Eringate, Old Burnhamthorpe",Pharmacy,Convenience Store,Beer Store,Electronics Store,Liquor Store,Café,Pizza Place,Cosmetics Shop,Dog Run,Farmers Market
3,Brockton,Coffee Shop,Bar,Vietnamese Restaurant,Clothing Store,Bakery,Fast Food Restaurant,Grocery Store,Big Box Store,Pizza Place,Candy Store
4,Business reply mail Processing Centre969 Eastern,Diner,Café,Coffee Shop,Bakery,Sandwich Place,Italian Restaurant,Gastropub,Juice Bar,Sushi Restaurant,Cheese Shop
5,Caledonia-Fairbanks,Park,Pharmacy,Market,Cosmetics Shop,Fast Food Restaurant,Women's Store,Concert Hall,Construction & Landscaping,Farmers Market,Falafel Restaurant
6,Canada Post Gateway Processing Centre,Pizza Place,Auto Workshop,Comic Shop,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop
7,Central Bay Street,Coffee Shop,Clothing Store,Café,Burger Joint,Bar,Tea Room,Italian Restaurant,Chinese Restaurant,Spa,Japanese Restaurant
8,Christie,Grocery Store,Park,Coffee Shop,Japanese Restaurant,Café,Candy Store,Italian Restaurant,Nightclub,Beer Store,Diner
9,"Clarks Corners, Sullivan",Coffee Shop,Hotel,Café,American Restaurant,Gastropub,Restaurant,Deli / Bodega,Gym,Tea Room,Japanese Restaurant


And finally, let's do the clustering!
using 7 clusters

In [292]:
temp = dFinal.drop('Neighborhood', 1)

Kcluster = KMeans(n_clusters=7, random_state=0).fit(temp)


Associating with the df:

In [293]:
dFinal['Cluster'] = Kcluster.labels_

#for key, value in nearVenuesDF['Neighborhood']:
 #   print(nearVenuesDF[key])

#(dFinal['Neighborhood'] == nearVenuesDF['neighborhood'])
#pd.concat([dFinal, nearVenuesDF], join='inner', axis=1)
dFinal=nearVenuesDF.join(dFinal.set_index('Neighborhood'), on='neighborhood')
dFinal.head()

,latitude,location,longitude,name,neighborhood,type,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster
0,43.752743,"Scarborough Village, Toronto",-79.192777,Via. Rail 651,Guildwood,Moving Target,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,43.752743,"Scarborough Village, Toronto",-79.192777,Poplar,Guildwood,Baseball Field,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,43.752743,"Scarborough Village, Toronto",-79.192777,VIA Rail Guildwood,Guildwood,Train Station,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Mama's Boys Burgers,East Birchmount Park,Burger Joint,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Shell,East Birchmount Park,Gas Station,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


And finally, let's see it graphically:

In [295]:
mapa = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(7)
ys = [i+x+(i*x)**2 for i in range(7)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(dFinal['latitude'], dFinal['longitude'], dFinal['neighborhood'], dFinal['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapa)
       
mapa

And the analisys by cluster:

In [306]:
dFinal[dFinal['Cluster'] == 0]

,latitude,location,longitude,name,neighborhood,type,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster
525,43.656898,"Downtown Toronto, Toronto",-79.376122,Page One Cafe,Garden District,Café,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
526,43.656898,"Downtown Toronto, Toronto",-79.376122,The Grand Hotel & Suites Toronto,Garden District,Hotel,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
527,43.656898,"Downtown Toronto, Toronto",-79.376122,Burrito Boyz,Garden District,Burrito Place,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
528,43.656898,"Downtown Toronto, Toronto",-79.376122,Tacos 101,Garden District,Taco Place,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
529,43.656898,"Downtown Toronto, Toronto",-79.376122,Hokkaido Ramen Santouka らーめん山頭火,Garden District,Ramen Restaurant,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
530,43.656898,"Downtown Toronto, Toronto",-79.376122,Citrus Restaurant & Lounge @ The Grand Hotel,Garden District,Lounge,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
531,43.656898,"Downtown Toronto, Toronto",-79.376122,Blaze Pizza,Garden District,Pizza Place,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
532,43.656898,"Downtown Toronto, Toronto",-79.376122,The Senator Restaurant,Garden District,Diner,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
533,43.656898,"Downtown Toronto, Toronto",-79.376122,Oakham Café,Garden District,Café,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
534,43.656898,"Downtown Toronto, Toronto",-79.376122,UNIQLO ユニクロ,Garden District,Clothing Store,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [305]:
dFinal[dFinal['Cluster'] == 1]

,latitude,location,longitude,name,neighborhood,type,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster
0,43.752743,"Scarborough Village, Toronto",-79.192777,Via. Rail 651,Guildwood,Moving Target,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,43.752743,"Scarborough Village, Toronto",-79.192777,Poplar,Guildwood,Baseball Field,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,43.752743,"Scarborough Village, Toronto",-79.192777,VIA Rail Guildwood,Guildwood,Train Station,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Mama's Boys Burgers,East Birchmount Park,Burger Joint,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Shell,East Birchmount Park,Gas Station,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,The Beer Store,East Birchmount Park,Beer Store,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Subway,East Birchmount Park,Sandwich Place,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Pizza Pizza,East Birchmount Park,Pizza Place,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Fu Yao Supermarket,East Birchmount Park,Grocery Store,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,43.711087,"Clairlea - Birchmount, Toronto",-79.269800,Pizza Hut,East Birchmount Park,Pizza Place,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [304]:
dFinal[dFinal['Cluster'] == 2]

,latitude,location,longitude,name,neighborhood,type,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster
699,43.647847,"Entertainment District, Toronto",-79.395880,Alo,Richmond,French Restaurant,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
700,43.647847,"Entertainment District, Toronto",-79.395880,Banh Mi Boys,Richmond,Sandwich Place,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
701,43.647847,"Entertainment District, Toronto",-79.395880,Yeti Nails & Spa,Richmond,Cosmetics Shop,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
702,43.647847,"Entertainment District, Toronto",-79.395880,Avenue Open Kitchen,Richmond,Diner,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
703,43.647847,"Entertainment District, Toronto",-79.395880,Come As You Are Co-operative,Richmond,Adult Boutique,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
704,43.647847,"Entertainment District, Toronto",-79.395880,Get Outside,Richmond,Shoe Store,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
705,43.647847,"Entertainment District, Toronto",-79.395880,Yoga Tree Downtown,Richmond,Yoga Studio,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
706,43.647847,"Entertainment District, Toronto",-79.395880,Fresh On Spadina,Richmond,Vegetarian / Vegan Restaurant,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
707,43.647847,"Entertainment District, Toronto",-79.395880,Strange Love,Richmond,Coffee Shop,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2
708,43.647847,"Entertainment District, Toronto",-79.395880,City Dance Corps,Richmond,Dance Studio,1,0,0,0,...,2,1,0,0,0,0,0,0,3,2


In [303]:
dFinal[dFinal['Cluster'] == 3]

,latitude,location,longitude,name,neighborhood,type,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster
351,43.658395,"Leslieville, Toronto",-79.338965,Leslieville Pumps,Studio District,Sandwich Place,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
352,43.658395,"Leslieville, Toronto",-79.338965,Te Aro,Studio District,Coffee Shop,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
353,43.658395,"Leslieville, Toronto",-79.338965,Ed's Real Scoop,Studio District,Ice Cream Shop,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
354,43.658395,"Leslieville, Toronto",-79.338965,Lady Marmalade,Studio District,Breakfast Spot,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
355,43.658395,"Leslieville, Toronto",-79.338965,Leslieville,Studio District,Neighborhood,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
356,43.658395,"Leslieville, Toronto",-79.338965,Mercury Espresso Bar,Studio District,Coffee Shop,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
357,43.658395,"Leslieville, Toronto",-79.338965,Queen Books,Studio District,Bookstore,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
358,43.658395,"Leslieville, Toronto",-79.338965,Hooked,Studio District,Fish Market,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
359,43.658395,"Leslieville, Toronto",-79.338965,Belmonte Raw Food And Juice Bar,Studio District,Juice Bar,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
360,43.658395,"Leslieville, Toronto",-79.338965,Sweat & Soda,Studio District,Café,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [302]:
dFinal[dFinal['Cluster'] == 4]

,latitude,location,longitude,name,neighborhood,type,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster
873,43.648159,"Financial District, Toronto",-79.377851,Hockey Hall Of Fame (Hockey Hall of Fame),Victoria Hotel,Museum,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
874,43.648159,"Financial District, Toronto",-79.377851,Equinox Bay Street,Victoria Hotel,Gym,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
875,43.648159,"Financial District, Toronto",-79.377851,Beerbistro,Victoria Hotel,Gastropub,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
876,43.648159,"Financial District, Toronto",-79.377851,Walrus Pub & Beer Hall,Victoria Hotel,Pub,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
877,43.648159,"Financial District, Toronto",-79.377851,Mos Mos Coffee,Victoria Hotel,Café,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
878,43.648159,"Financial District, Toronto",-79.377851,Ki Modern Japanese + Bar,Victoria Hotel,Japanese Restaurant,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
879,43.648159,"Financial District, Toronto",-79.377851,Dineen Coffee,Victoria Hotel,Café,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
880,43.648159,"Financial District, Toronto",-79.377851,The Omni King Edward Hotel,Victoria Hotel,Hotel,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
881,43.648159,"Financial District, Toronto",-79.377851,Design Exchange,Victoria Hotel,Art Gallery,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
882,43.648159,"Financial District, Toronto",-79.377851,Hogtown Smoke,Victoria Hotel,Food Truck,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4


In [310]:
dFinal[dFinal['Cluster'] == 5]

,latitude,location,longitude,name,neighborhood,type,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster
38,43.647361,"Financial District, Toronto",-79.381123,Canoe,"Clarks Corners, Sullivan",Restaurant,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
39,43.647361,"Financial District, Toronto",-79.381123,Equinox Bay Street,"Clarks Corners, Sullivan",Gym,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
40,43.647361,"Financial District, Toronto",-79.381123,Walrus Pub & Beer Hall,"Clarks Corners, Sullivan",Pub,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
41,43.647361,"Financial District, Toronto",-79.381123,The Fairmont Royal York,"Clarks Corners, Sullivan",Hotel,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
42,43.647361,"Financial District, Toronto",-79.381123,Adelaide Club Toronto,"Clarks Corners, Sullivan",Gym / Fitness Center,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
43,43.647361,"Financial District, Toronto",-79.381123,Mos Mos Coffee,"Clarks Corners, Sullivan",Café,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
44,43.647361,"Financial District, Toronto",-79.381123,King Taps,"Clarks Corners, Sullivan",Gastropub,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
45,43.647361,"Financial District, Toronto",-79.381123,Maman,"Clarks Corners, Sullivan",Café,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
46,43.647361,"Financial District, Toronto",-79.381123,Ki Modern Japanese + Bar,"Clarks Corners, Sullivan",Japanese Restaurant,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
47,43.647361,"Financial District, Toronto",-79.381123,Pilot Coffee Roasters,"Clarks Corners, Sullivan",Coffee Shop,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5


In [311]:
dFinal[dFinal['Cluster'] == 6]

,latitude,location,longitude,name,neighborhood,type,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster
1282,43.6528,"Kensington, Toronto",-79.400137,Seven Lives - Tacos y Mariscos,Del Ray,Mexican Restaurant,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1283,43.6528,"Kensington, Toronto",-79.400137,Kid Icarus,Del Ray,Arts & Crafts Store,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1284,43.6528,"Kensington, Toronto",-79.400137,The Moonbean Cafe,Del Ray,Café,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1285,43.6528,"Kensington, Toronto",-79.400137,Cold Tea,Del Ray,Bar,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1286,43.6528,"Kensington, Toronto",-79.400137,Essence of Life Organics,Del Ray,Organic Grocery,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1287,43.6528,"Kensington, Toronto",-79.400137,El Rey,Del Ray,Cocktail Bar,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1288,43.6528,"Kensington, Toronto",-79.400137,Jimmy's Coffee,Del Ray,Café,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1289,43.6528,"Kensington, Toronto",-79.400137,FIKA Cafe,Del Ray,Café,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1290,43.6528,"Kensington, Toronto",-79.400137,Banh Mi Nguyen Huong,Del Ray,Vietnamese Restaurant,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
1291,43.6528,"Kensington, Toronto",-79.400137,People's Eatery 眾源酒家,Del Ray,Chinese Restaurant,0,0,0,0,...,6,0,0,4,1,0,0,0,0,6
